In [1]:
from bs4 import BeautifulSoup

In [2]:
soup_raw = BeautifulSoup(open("MSFT 10K 2018 630.htm"), 'html.parser')

In [3]:
text = soup_raw.get_text()
text = text.replace("\n"," ")
text = text.replace("\xa0"," ")

In [4]:
text[:100]

' 10-K 1 msft-10k_20180630.htm 10-K      msft-10k_20180630.htm         UNITED STATES  SECURITIES AND '

In [5]:
def find_nth(string, substring, n):
    if (n == 1):
        return string.find(substring)
    else:
        return string.find(substring, find_nth(string, substring, n - 1) + 1)

In [6]:
index = find_nth(text,"Item",4)

In [7]:
text[index-30:index+30]

' 17                           Item 1B.     Unresolved Staff '

In [8]:
headers = [] 

for i in range(2,25):
    index = find_nth(text,"Item",i)
    header = text[index:index+8]
    j=0
    while(text[index+8+j].isdigit() == False):
        j+=1 
    
    # handle item 16 Form 10-K
    
    header += text[index+8:index+8+j]
    
    if ("." in text[index+5:index+9]):
        headers.append(header)
    
print(headers)
print("------------------------Handle the last entry of Form 10-k")

['Item 1.     Business     ', 'Item 1A.     Risk Factors     ', 'Item 1B.     Unresolved Staff Comments     ', 'Item 2.     Properties     ', 'Item 3.     Legal Proceedings     ', 'Item 4.     Mine Safety Disclosures     ', 'Item 5.     Market for Registrant’s Common Equity, Related Stockholder Matters, and Issuer Purchases of Equity Securities     ', 'Item 6.     Selected Financial Data     ', 'Item 7.     Management’s Discussion and Analysis of Financial Condition and Results of Operations     ', 'Item 7A.     Quantitative and Qualitative Disclosures about Market Risk     ', 'Item 8.     Financial Statements and Supplementary Data     ', 'Item 9.     Changes in and Disagreements with Accountants on Accounting and Financial Disclosure     ', 'Item 9A.     Controls and Procedures     ', 'Item 9B.     Other Information     ', 'Item 10.     Directors, Executive Officers and Corporate Governance     ', 'Item 11.     Executive Compensation     ', 'Item 12.     Security Ownership of Certain

In [131]:
index = find_nth(text,"Item",17)
print(text[index-30:index+30])

100                           Item 11.     Executive Compens


In [19]:
items = ["Item 1.", "Item 1A.", "Item 1B.", "Item 2.", "Item 3.", 
         "Item 4.","Item 5.", "Item 6.","Item 7.","Item 7A.","Item 8.",
         "Item 9.","Item 9A.","Item 9B.","Item 10.","Item 11.","Item 12.",
         "Item 13.","Item 14.", "Item 15.","Item 16.","Signatures" ]

def parse_10k(text,items):
    sections = {}
    for i in range(len(items)-1):
        section_start_index = find_nth(text,items[i],2)
        if (section_start_index == -1):
            # Also try upper case 
            print("trying upper case..{}".format(items[i].upper()))
            section_start_index = find_nth(text, items[i].upper(), 1)

        section_end_index = find_nth(text,items[i+1],2) 
        if (section_end_index == -1):
            # Also try upper case 
            section_end_index = find_nth(text, items[i+1].upper(), 1)
        
        if ( (section_start_index != -1) and (section_end_index != -1)  ):
            sections[items[i]] = text[section_start_index:section_end_index]
        else:
            print("Something went wrong at iteration {}".format(i))
            print(section_start_index,section_end_index)
            sections[items[i]] = ""
    return sections 
    

In [16]:
index = find_nth(text, "ITEM", 1)
print(text[index-15:index+15])

wise.  PART I  ITEM 1. BUSINES


In [20]:
sections_msft = parse_10k(text, items)

trying upper case..ITEM 1.
trying upper case..ITEM 1A.
trying upper case..ITEM 1B.
trying upper case..ITEM 2.
trying upper case..ITEM 3.
trying upper case..ITEM 4.
trying upper case..ITEM 5.
trying upper case..ITEM 6.
trying upper case..ITEM 7.
trying upper case..ITEM 7A.
trying upper case..ITEM 8.
trying upper case..ITEM 9.
trying upper case..ITEM 9B.
trying upper case..ITEM 10.
trying upper case..ITEM 11.
trying upper case..ITEM 12.
trying upper case..ITEM 13.
trying upper case..ITEM 14.
trying upper case..ITEM 15.
trying upper case..ITEM 16.


In [36]:
for key, item in sections_msft.items():
    print("key is {}".format(key))
    print(item[:150])
    print("--")
    print(item[-150:])
    print("------------------------")

key is Item 7A.
ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK  RISKS  We are exposed to economic risk from foreign exchange rates, interest rate
--
se in equity market prices        (140  )         (323  )      Fair Value                                                        50  PART II Item 8   
------------------------
key is Item 1A.
ITEM 1A. RISK FACTORS  Our operations and financial results are subject to various risks and uncertainties, including those described below, that coul
--
ged, it could negatively impact our revenues or margins, or ability to attract the most highly qualified employees.     28  PART I Item 1B, 2, 3, 4   
------------------------
key is Item 1B.
ITEM 1B. UNRESOLVED STAFF COMMENTS  We have received no written comments regarding our periodic or current reports from the staff of the Securities an
--
 staff of the Securities and Exchange Commission that were issued 180 days or more preceding the end of our fiscal year 2018 that remain u

In [61]:
def find_nth(string, substring, n):
    if (n == 1):
        return string.find(substring)
    else:
        return string.find(substring, find_nth(string, substring, n - 1) + 1)
items = ["Item 1.", "Item 1A.", "Item 1B.", "Item 2.", "Item 3.", 
         "Item 4.","Item 5.", "Item 6.","Item 7.","Item 7A.","Item 8.",
         "Item 9.","Item 9A.","Item 9B.","Item 10.","Item 11.","Item 12.",
         "Item 13.","Item 14.", "Item 15.","Item 16.","Signatures" ]

def parse_10k(text,items):
    sections = {}
    for i in range(len(items)-1):
        section_start_index = find_nth(text,items[i],2)
        if (section_start_index == -1):
            # Also try upper case 
            print("trying upper case..{}".format(items[i].upper()))
            section_start_index = find_nth(text, items[i].upper(), 1)

        section_end_index = find_nth(text,items[i+1],2) 
        if (section_end_index == -1):
            # Also try upper case 
            section_end_index = find_nth(text, items[i+1].upper(), 1)
        
        if ( (section_start_index != -1) and (section_end_index != -1)  ):
            sections[items[i]] = text[section_start_index:section_end_index]
        else:
            print("Something went wrong for {}".format(items[i]))
            print(section_start_index,section_end_index)
            sections[items[i]] = ""
            
        # Process for 10-Ks without Item 16 
        if ( (items[i+1] == "Item 16.") and (section_end_index == -1 ) ):
            # Just skip to Signatures
            section_end_index = find_nth(text,items[i+2],2)
            
            # try upper case
            if (section_end_index == -1):
                section_end_index = find_nth(text,items[i+2].upper(),1)
           
            if ( (section_start_index != -1) and (section_end_index != -1)  ):
                sections[items[i]] = text[section_start_index:section_end_index]
            else:
                print("Something went wrong for {}".format(items[i]))
                print(section_start_index,section_end_index)
                sections[items[i]] = ""
                break 
        
    return sections 
    
    
def process_10_k(html_file):
    soup_raw = BeautifulSoup(open(html_file), 'html.parser')
    text = soup_raw.get_text()
    text = text.replace("\n"," ")
    text = text.replace("\xa0"," ") 
    
    sections = parse_10k(text, items) 
    return sections

def check_correct(sections):
    for key, item in sections.items():
        print("key is {}".format(key))
        print(item[:150])
        print("--")
        print(item[-150:])
        print("------------------------")

In [59]:
msft = process_10_k("MSFT 10K 2018 630.htm")
msft["Item 1."][:100]

trying upper case..ITEM 1.
trying upper case..ITEM 1A.
trying upper case..ITEM 1B.
trying upper case..ITEM 2.
trying upper case..ITEM 3.
trying upper case..ITEM 4.
trying upper case..ITEM 5.
trying upper case..ITEM 6.
trying upper case..ITEM 7.
trying upper case..ITEM 7A.
trying upper case..ITEM 8.
trying upper case..ITEM 9.
trying upper case..ITEM 9B.
trying upper case..ITEM 10.
trying upper case..ITEM 11.
trying upper case..ITEM 12.
trying upper case..ITEM 13.
trying upper case..ITEM 14.
trying upper case..ITEM 15.
trying upper case..ITEM 16.


'ITEM 1. BUSINESS  GENERAL  Embracing Our Future Microsoft is a technology company whose mission is t'

In [60]:
amzn = process_10_k("AMZN 10K 2007.html")
amzn["Item 1."][:100]

Something went wrong for Item 15.
269754 -1
trying upper case..ITEM 16.
Something went wrong for Item 16.
-1 274689


'Item 1.    Business   This Annual Report on Form 10-K and the documents incorporated herein by refer'

In [62]:
check_correct(amzn)

key is Item 7A.
Item 7A.    Quantitative and Qualitative Disclosure About Market Risk  We are exposed to market risk for the effect of interest rate changes, foreign 
--
s would result in a corresponding decline in total fair value of approximately $10 million, $15 million, and $25 million.    42    Table of Contents  
------------------------
key is Item 1A.
Item 1A.    Risk Factors  Please carefully consider the following risk factors. If any of the following risks occur, our business, financial condition
--
n unlawful manner, or violating the proprietary rights of others, and could face civil or criminal liability for unlawful activities by our sellers.  
------------------------
key is Item 1B.
Item 1B.     Unresolved Staff Comments  None.  
--
Item 1B.     Unresolved Staff Comments  None.  
------------------------
key is Item 6.
Item 6.     Selected Consolidated Financial Data   The following selected consolidated financial data should be read in conjunction with the consolida
--